# New York CitiBike

#### Description
The New York CitiBike program is the largest bicycle sharing service in the United States.  Each month, the city makes the raw data of all rides taken in that month available.  This data is available for developers, but the city itself seems to not publish analysis.

Ultimately, this project aims to create visualizations and summaries using the Tableau software package.  However, these individual monthly files are of course large and likely unworkable in Tableau.  Therefore, this notebook is used to perform cleaning operations to produce csv files that can be easily used in Tableau.

Ultimately, the goal is to summarize ride data for the past two years (2017/2018).

#### Load Packages

First, we must load in useful packages such as os and Pandas.

In [1]:
#import packages
import os
import numpy as np
import pandas as pd

#### Save file reference

We will save a reference to the data file.

In [2]:
#create file path
filepath = os.path.join('..', 'data')

#### Explore one file

In the interests of development, we will start with exploring one file, hoping to develop a method (or methods) that summarizes data for a single month.

In [3]:
#get one file
file = os.path.join(filepath, 'JC-201812-citibike-tripdata.csv')

In [4]:
#read in file
test_data = pd.read_csv(file)

In [5]:
#take a look
test_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,170,2018-12-01 00:03:53.3860,2018-12-01 00:06:43.7160,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,26261,Subscriber,1975,1
1,172,2018-12-01 00:47:13.7340,2018-12-01 00:50:05.8160,3183,Exchange Place,40.716247,-74.033459,3481,York St,40.716490,-74.041050,26175,Subscriber,1988,1
2,247,2018-12-01 00:48:57.6960,2018-12-01 00:53:05.6780,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,29255,Subscriber,1990,1
3,406,2018-12-01 01:33:53.6630,2018-12-01 01:40:39.7480,3183,Exchange Place,40.716247,-74.033459,3638,Washington St,40.724294,-74.035483,29275,Subscriber,1995,1
4,354,2018-12-01 06:26:52.5120,2018-12-01 06:32:46.5170,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,29658,Subscriber,1976,1


#### Check formatting of older data

We should also test the oldest file to see if there has been any change in how the files are formatted.

In [6]:
#create file path
old_file = os.path.join(filepath, 'JC-201701-citibike-tripdata.csv')
old_data = pd.read_csv(old_file)
old_data.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,148,2017-01-01 00:21:32,2017-01-01 00:24:01,3276,Marin Light Rail,40.714584,-74.042817,3185,City Hall,40.717733,-74.043845,24575,Subscriber,1983.0,1
1,1283,2017-01-01 00:24:35,2017-01-01 00:45:58,3183,Exchange Place,40.716247,-74.033459,3198,Heights Elevator,40.748716,-74.040443,24723,Subscriber,1978.0,1
2,372,2017-01-01 00:38:19,2017-01-01 00:44:31,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,24620,Subscriber,1989.0,1
3,1513,2017-01-01 00:38:37,2017-01-01 01:03:50,3194,McGinley Square,40.725340,-74.067622,3271,Danforth Light Rail,40.692640,-74.088012,24668,Subscriber,1961.0,1
4,639,2017-01-01 01:47:52,2017-01-01 01:58:31,3183,Exchange Place,40.716247,-74.033459,3203,Hamilton Park,40.727596,-74.044247,26167,Subscriber,1993.0,1


Looking at this, we see that the main change is that there is now subsecond level data.  This seems to have changed for 2018 data vs. 2017 (by physically looking at files).  As we likely have no need of subsecond precision, we will strip it off if there is a need.

#### Column List
We should also check the column names.

In [7]:
test_data.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [8]:
old_data.columns

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')

We see that we have the same columns but slightly different name formatting.  We will assign the names to be the same.

In [9]:
#reassign columns
old_data.columns = test_data.columns
old_data.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

#### Dates and Times

We need to do some string splits to get the dates and times in an easier format.

In [16]:
#get dates
test_data['start_date'] = test_data['starttime'].str.split(' ', expand = True).iloc[:, 0]
test_data['stop_date'] = test_data['stoptime'].str.split(' ', expand = True).iloc[:, 0]

In [18]:
#get times (remove decimal places)
test_data['start_time'] = test_data['starttime'].str.split(' ', expand = True).iloc[:, 1].str.split('.', 
                                                                                            expand = True).iloc[:, 0]
test_data['stop_time'] = test_data['stoptime'].str.split(' ', expand = True).iloc[:, 1].str.split('.', 
                                                                                            expand = True).iloc[:, 0]

In [19]:
#look at result
test_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_date,stop_date,start_time,stop_time
0,170,2018-12-01 00:03:53.3860,2018-12-01 00:06:43.7160,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,26261,Subscriber,1975,1,2018-12-01,2018-12-01,00:03:53,00:06:43
1,172,2018-12-01 00:47:13.7340,2018-12-01 00:50:05.8160,3183,Exchange Place,40.716247,-74.033459,3481,York St,40.716490,-74.041050,26175,Subscriber,1988,1,2018-12-01,2018-12-01,00:47:13,00:50:05
2,247,2018-12-01 00:48:57.6960,2018-12-01 00:53:05.6780,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,29255,Subscriber,1990,1,2018-12-01,2018-12-01,00:48:57,00:53:05
3,406,2018-12-01 01:33:53.6630,2018-12-01 01:40:39.7480,3183,Exchange Place,40.716247,-74.033459,3638,Washington St,40.724294,-74.035483,29275,Subscriber,1995,1,2018-12-01,2018-12-01,01:33:53,01:40:39
4,354,2018-12-01 06:26:52.5120,2018-12-01 06:32:46.5170,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,29658,Subscriber,1976,1,2018-12-01,2018-12-01,06:26:52,06:32:46


#### Station List

One task we have is to extract a list of all stations.  We will attempt this with the most recent data.

We would like counts of the people who leave from each station.

In [34]:
#group by start station id and start_date
grouped_by_start = test_data.groupby(['start station id'])

In [36]:
#look at names
start_stations = grouped_by_start['start station name'].first()
start_stations

start station id
3183             Exchange Place
3184                Paulus Hook
3185                  City Hall
3186              Grove St PATH
3187                  Warren St
3191                   Union St
3192         Liberty Light Rail
3193               Lincoln Park
3194            McGinley Square
3195                    Sip Ave
3196             Riverview Park
3198           Heights Elevator
3199               Newport Pkwy
3201                     Dey St
3202               Newport PATH
3203              Hamilton Park
3205          JC Medical Center
3206                    Hilltop
3207                Oakland Ave
3209               Brunswick St
3210             Pershing Field
3211                 Newark Ave
3212            Christ Hospital
3213             Van Vorst Park
3214           Essex Light Rail
3220          5 Corners Library
3225      Baldwin at Montgomery
3267               Morris Canal
3268             Lafayette Park
3269            Brunswick & 6th
3270            Jersey 

In [39]:
#now lets take a look at the counts
start_station_counts = grouped_by_start['start station name'].count()
start_station_counts

start station id
3183     904
3184     355
3185     481
3186    2535
3187     413
3191      56
3192     332
3193     145
3194     357
3195    1199
3196      82
3198     133
3199     454
3201      46
3202     775
3203    1185
3205     305
3206     141
3207     315
3209     480
3210     338
3211     516
3212     102
3213     480
3214     319
3220     116
3225     259
3267     485
3268     111
3269     547
3270     513
3272     464
3273     330
3275     408
3276     541
3277      77
3278     435
3279     398
3280     224
3281     158
3426       4
3481     202
3483     263
3638     383
3639     688
3640     423
3677      69
3678     177
3679     145
3681     285
3694      52
Name: start station name, dtype: int64

It is also useful to look at end station counts, so we will group by that as well and compute counts.

In [40]:
#group by end station id
grouped_by_end = test_data.groupby(['end station id'])

#compute counts
end_station_counts = grouped_by_end['end station name'].count()

#display counts
end_station_counts

end station id
254        1
304        1
459        1
482        2
3002       2
3183    1126
3184     378
3185     574
3186    3187
3187     462
3191      70
3192     366
3193     175
3194     307
3195    1139
3196      74
3198     127
3199     454
3201      82
3202     772
3203    1025
3205     282
3206     108
3207     179
3209     405
3210     298
3211     606
3212      91
3213     404
3214     331
3220      93
3225     235
3267     445
3268     105
3269     435
3270     366
3272     411
3273     448
3275     426
3276     489
3277      65
3278     414
3279     346
3280     223
3281     139
3426       7
3481     192
3483     235
3638     412
3639     701
3640     381
3677      39
3678     126
3679     123
3681     276
3694      44
Name: end station name, dtype: int64

#### Joining Counts

It is to our advantage to combine these counts into one dataframe.  First, we will make individual data frames and then join them.  We will need to set nonoccurring counts to 0.

In [43]:
#Create a dataframe with start counts
start_station_df = pd.DataFrame({'start station count': start_station_counts})

#create a dataframe with end counts
end_station_df = pd.DataFrame({'end station count': end_station_counts})

In [45]:
#reset indices
start_station_df.reset_index(inplace = True)
end_station_df.reset_index(inplace = True)

In [47]:
#rename columns
start_station_df.columns = ['station id', 'start station count']
end_station_df.columns = ['station id', 'end station count']

In [53]:
#join on station id
stations_df = start_station_df.merge(end_station_df, on = 'station id', how = 'outer')

#make all NaNs zero
stations_df = stations_df.fillna(0)

#look at data frame
stations_df.head()

,station id,start station count,end station count
0,3183,904.0,1126
1,3184,355.0,378
2,3185,481.0,574
3,3186,2535.0,3187
4,3187,413.0,462


#### Summary of subscriber types, age, user type, trip duration

We want to create a monthly summary of various variables like user type (24-hour or 3-day pass), gender, trip duration (minutes), and user age

In [57]:
#trip duration
#convert trip duration to minutes for ease of use
test_data['duration_min'] = test_data['tripduration']/60

#describe it
duration_description = test_data['duration_min'].describe()

duration_description

count    20205.000000
mean         9.350120
std        103.255582
min          1.016667
25%          3.533333
50%          5.066667
75%          7.933333
max       8938.483333
Name: duration_min, dtype: float64

In [58]:
#age
#calculate estimated age as year - birthyear
test_data['age'] = 2018 - test_data['birth year']

#describe it
age_description = test_data['age'].describe()

age_description

count    20205.000000
mean        36.625637
std         10.089665
min         16.000000
25%         29.000000
50%         34.000000
75%         42.000000
max         77.000000
Name: age, dtype: float64

In [65]:
#gender
#get value counts (0 for unknown, 1 for male, 2 for female)
gender_counts = test_data['gender'].value_counts()

gender_counts

1    15161
2     4325
0      719
Name: gender, dtype: int64

In [70]:
#user type
#get value counts (subscribers are annual pass holders, customers hold 24-hour passes or 3-day passes)
user_counts = test_data['usertype'].value_counts()

user_counts

Subscriber    19614
Customer        591
Name: usertype, dtype: int64

We now have all we need to make a dataframe.  However, we obviously only need some values (for instance we don't need the total count twice).  Also, the minimum and maximum trip durations seem somewhat flukey (problems starting or forgetting to return bike), so we will ignore them.  

In [74]:
#create a monthly summary dataframe
summary_df = pd.DataFrame({'Month': ['2018-12'], 'Number of Rides': [duration_description['count']],
                          'Mean Duration': [duration_description['mean']], '1Q Duration': [duration_description['25%']],
                          '3Q Duration': [duration_description['75%']], 'Median Duration': [duration_description['50%']],
                          'Mean Age': [age_description['mean']], 'StdDev Age': [age_description['std']],
                          'Min Age': [age_description['min']], 'Max Age': [age_description['max']],
                          '1Q Age': [age_description['25%']], '3Q Age': [age_description['75%']],
                          'Median Age': [age_description['50%']], 'Unknown Gender Users': [gender_counts[0]],
                          'Male Users': [gender_counts[1]], 'Female Users': [gender_counts[2]],
                          'Short-Term Pass Uses': [user_counts['Customer']], 
                           'Annual Pass Uses': [user_counts['Subscriber']]})

In [75]:
#look at dataframe
summary_df

,Month,Number of Rides,Mean Duration,1Q Duration,3Q Duration,Median Duration,Mean Age,StdDev Age,Min Age,Max Age,1Q Age,3Q Age,Median Age,Unknown Gender Users,Male Users,Female Users,Short-Term Pass Uses,Annual Pass Uses
0,2018-12,20205.0,9.35012,3.533333,7.933333,5.066667,36.625637,10.089665,16.0,77.0,29.0,42.0,34.0,719,15161,4325,591,19614


#### Starting Hour of the Day
We would like to know what time of day people typically use the bikes.

To do so, we will strip out the hour and group by it.

In [76]:
#calculate hour
test_data['hour'] = test_data['start_time'].str.split(':', expand = True).iloc[:, 0]

In [77]:
#group by hour
grouped_by_hour = test_data.groupby('hour')

In [78]:
#get counts
hour_counts = grouped_by_hour['start_time'].count()

In [80]:
#create dataframe
hour_df = pd.DataFrame(hour_counts)

In [81]:
#reset index
hour_df.reset_index(inplace = True)

In [83]:
#rename columns and show
hour_df.columns = ['Hour', 'Count']
hour_df.head()

,Hour,Count
0,00,178
1,01,109
2,02,56
3,03,35
4,04,69
